In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile

In [2]:
input_folder = 'Data'
output_folder = 'Spectrograms'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [4]:
for file in os.listdir(input_folder):
    if(file.endswith(".wav")):
        create_spectrogram(file, output_folder, input_folder)


In [3]:
def create_spectrogram(file, ofolder_path, ifolder_path):
    
    file_path = os.path.join(ifolder_path, file)

    sample_rate, samples = wavfile.read(file_path)
    frequencies, times, spectrogram = signal.spectrogram(samples, sample_rate)

    base_name = os.path.splitext(os.path.basename(file))[0]

    plt.figure(figsize=(10,5))
    #Deleting cmap='gray' speeds the creation of spectrogram by 5 times, but its colorful
    plt.pcolormesh(times, frequencies, np.log(spectrogram), shading='gouraud', cmap='gray')
    plt.title(base_name)
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')

    new_path = os.path.join(ofolder_path, base_name + '.jpg')

    plt.savefig(new_path)
    plt.close()